In [16]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [17]:
# reading the dataset

df = pd.read_csv(r'..\Data\house_prices.csv')

In [18]:
# looking at the data

display(df.head(5),df.sample(5),df.tail(5))

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
6349,Built-up Area,Ready To Move,Brookefield,2 BHK,Bhiya N,1100,2.0,1.0,55.0
12520,Plot Area,Ready To Move,BTM 2nd Stage,8 Bedroom,NaN,1500,6.0,0.0,270.0
6210,Super built-up Area,Ready To Move,BTM Layout,3 BHK,C ensnd,1458,3.0,2.0,79.0
11410,Super built-up Area,18-Jul,Ramagondanahalli,3 BHK,ViistLa,1635,3.0,1.0,57.0
3850,Built-up Area,Ready To Move,Nagaraja Garden,2 BHK,NaN,1256,2.0,1.0,62.8


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.0
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.0
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.0
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.0
13319,Super built-up Area,Ready To Move,Doddathoguru,1 BHK,NaN,550,1.0,1.0,17.0
